In [ ]:
#  #########################################################################
#  #                           IN THE NAME OF ALLAH                        #
#  #     STEEL GABLE FRAME SENSITIVITY ANALYSIS WEB HEIGHT NONPRISMATIC    #
#  #              I SECTION COLUMN WITH FINITE PRISMATIC COLUMN            #
#  #                    WITH NONLINEAR PUSHOVER ANALYSIS                   #
#  #                         NEWTON-RAPHSON METHOD                         #
#  #    MODELING OF NONPRISMATIC ELEMENT WITH MULTI PRISMATIC ELEMENTS     #
#  #-----------------------------------------------------------------------#
#  #              THIS PROGRAM WRITTEN BY SALAR DELAVAR QASHQAI            #
#  #                   EMAIL: salar.d.ghashghaei@gmail.com                 #
#  #########################################################################

In [ ]:
#import the os module
import os
import math
import time

In [ ]:
#to create a directory at specified path with name "Data"
os.mkdir('C:\\OPENSEESPY_SALAR')
#this will create the directory with name 'Data' and will update it when we rerun the analysis,
# otherwise we have to keep deleting the old 'Data' Folder
dir = "C:\\OPENSEESPY_SALAR\\OPENSEESPY_DATA"
if not os.path.exists(dir):
    os.makedirs(dir)

In [ ]:
def MAXABS_FUN(DATA_FILE, COLUMN, I):
    import numpy as np
    # Read and process displacement data
    NameFiles = DATA_FILE
    filename = f"{NameFiles}_{I}.txt"
    D = np.loadtxt(filename)
    #print(D)
    MAXABS = np.max(np.abs([D[:, COLUMN]]))
    #print("MAX. ABS. :", MAXABS)
    return MAXABS
# -----------------------------------------------
def PLOT_2D(X, Y, Xfit, Yfit, XLABEL, YLABEL, TITLE, COLOR, Z):
    import matplotlib.pyplot as plt
    plt.figure(figsize=(12, 8))
    if Z == 1:
        # Plot 1 line
        plt.plot(X, Y,color=COLOR)
        plt.xlabel(XLABEL)
        plt.ylabel(YLABEL)
        plt.title(TITLE)
        plt.grid(True)
        plt.show()
    if Z == 2:
        # Plot 2 lines
        plt.plot(X, Y, Xfit, Yfit, 'r--', linewidth=3)
        plt.title(TITLE)
        plt.xlabel(XLABEL)
        plt.ylabel(YLABEL)
        plt.legend(['curve', 'bilinear fitted'], loc='lower right')
        plt.grid(True)
        plt.show()
#--------------------------------------------
def HISROGRAM_BOXPLOT(X, HISTO_COLOR, LABEL):
    import numpy as np
    import matplotlib.pyplot as plt
    X = np.array(X)
    print("-------------------------")
    from scipy.stats import skew, kurtosis
    MINIMUM = np.min(X)
    MAXIMUM = np.max(X)
    #MODE = max(set(X), key=list(X).count)
    MEDIAN = np.quantile(X, .50)#q2
    MEAN = np.mean(X)
    STD = np.std(X)
    q1 = np.quantile(X, .25)
    q3 = np.quantile(X, .75)
    SKEW = skew(X)
    KURT = kurtosis(X)
    #SKEW = (MEAN - MODE) / STD
    #KURT = (np.mean((X - MEAN)**4) / STD**4)
    # Estimate confidence intervals of the output variable
    lower_bound = np.quantile(X, .05)
    upper_bound = np.quantile(X, .95)
    print("Box-Chart Datas: ")
    print(f'Minimum: {MINIMUM:.4e}')
    print(f'First quartile: {q1:.4e}')
    #print(f'Mode: {MODE:.4e}')
    print(f'Median: {MEDIAN:.4e}')
    print(f'Mean: {MEAN:.4e}')
    print(f'Std: {STD:.4e}')
    print(f'Third quartile: {q3:.4e}')
    print(f'Maximum: {MAXIMUM :.4e}')
    print(f'Skewness: {skew(X) :.4e}')
    print(f'kurtosis: {kurtosis(X) :.4e}')
    print(f"90% Confidence Interval: ({lower_bound:.4e}, {upper_bound:.4e})")
    print("-------------------------")

    plt.figure(figsize=(10,6))
    # Plot histogram of data
    count, bins, ignored = plt.hist(X, bins=100, color=HISTO_COLOR, density=True, align='mid')#, edgecolor="black"
    
    # Plot lognormal PDF
    x = np.linspace(min(bins), max(bins), 10000)
    pdf = (np.exp(-(x - MEAN)**2 / (2 * STD**2)) / (STD * np.sqrt(2 * np.pi)))
    plt.plot(x, pdf, linewidth=2, color='r', label="Normal PDF")
    
    # Plot vertical lines for risk measures
    plt.axvline(q1, color="black", linestyle="--", label=f"Quantile 0.25: {q1:.4e}")
    plt.axvline(MEDIAN, color="green", linestyle="--", label=f"Median: {MEDIAN:.4e}")
    plt.axvline(q3, color="black", linestyle="--", label=f"Quantile 0.75: {q3:.4e}")
    #plt.axvline(MODE, color="purple", linestyle="--", label=f"Mode: {MODE:.4e}")
    plt.axvline(MEAN, color="red", linestyle="--", label=f"Mean: {MEAN:.4e}")
    plt.axvline(MEAN-STD, color="blue", linestyle="--", label=f"Mean-Std: {MEAN-STD:.4e}")
    plt.axvline(MEAN+STD, color="blue", linestyle="--", label=f"Mean+Std: {MEAN+STD:.4e}")
    plt.xlabel(LABEL)
    plt.ylabel("Frequency")
    prob = np.sum(X > 0) / len(X)
    plt.title(f"Histogram - Probability of Positive {LABEL} is {100*prob:.2f} %")
    plt.legend()
    #plt.grid()
    plt.show()

    #Plot boxplot with outliers
    plt.figure(figsize=(10,6))
    plt.boxplot(X, vert=0)
    # Write the quartile data on the chart
    plt.text(q1, 1.05, f" Q1: {q1:.4e}")
    plt.text(MEDIAN, 1.1, f" Q2: {MEDIAN:.4e}")
    plt.text(q3, 1.05, f" Q3: {q3:.4e}")
    #plt.text(MODE, 1.15, f" Mode: {MODE:.4e}")
    
    #plt.text(MEAN, 0.9, f" Mean: {MEAN:.4e}")
    #plt.text(MEAN-STD, 0.9, f" Mean-Std: {MEAN-STD:.4e}")
    #plt.text(MEAN+STD, 0.9, f" Mean+Std: {MEAN+STD:.4e}")
    plt.scatter(MEAN, 1, color="red", marker="+", s=200, label=f"Mean: {MEAN:.4e}")
    plt.scatter(MEAN-STD, 1, color="green", marker="X", s=200, label=f"Mean-Std: {MEAN-STD:.4e}")
    plt.scatter(MEAN+STD, 1, color="blue", marker="*", s=200, label=f"Mean+Std:  {MEAN+STD:.4e}")
    plt.xlabel(LABEL)
    plt.ylabel("Data")
    plt.title(f"Boxplot of {LABEL}")
    plt.legend()
    plt.grid()
    plt.show()
# -----------------------------------------------    
def PLOT_TIME_HIS(DTH, VTH, ATH, BTH):
    ## PLOT THE DATA
    import matplotlib.pyplot as plt
    # Create a 1x3 grid of subplots
    fig, axs = plt.subplots(4, 1, figsize=(14, 14))

    # Plot Displacement
    axs[0].plot(DTH, label='Displacement')
    axs[0].set_title(f'Last Analysis Displacement Time History - Max(Abs): {np.max(np.abs(DTH)):.6f}')
    axs[0].set_xlabel('Time')
    axs[0].set_ylabel('Displacement')
    axs[0].grid()

    # Plot Velocity
    axs[1].plot(VTH, label='Velocity', color='purple')
    axs[1].set_title(f'Last Analysis Velocity Time History - Max(Abs): {np.max(np.abs(ATH)):.6f}')
    axs[1].set_xlabel('Time')
    axs[1].set_ylabel('Velocity')
    axs[1].grid()

    # Plot Acceleration
    axs[2].plot(ATH, label='Acceleration', color='green')
    axs[2].set_title(f'Last Analysis Acceleration Time History - Max(Abs): {np.max(np.abs(ATH)):.6f}')
    axs[2].set_xlabel('Time')
    axs[2].set_ylabel('Acceleration')
    axs[2].grid()
    
    # Plot Base Shear
    axs[3].plot(BTH, label='Base Shear', color='orange')
    axs[3].set_title(f'Last Analysis Base Shear Time History - Max(Abs): {np.max(np.abs(BTH)):.6f}')
    axs[3].set_xlabel('Time')
    axs[3].set_ylabel('Base Shear')
    axs[3].grid()

    # Adjust layout
    plt.tight_layout()
    plt.show()    
# -----------------------------------------------
def OUTPUT_SECOND_COLUMN(X, COLUMN, I):
    import numpy as np
    # Time History
    filename = f"C:\OPENSEESPY_SALAR\OPENSEESPY_DATA\\{X}_{I}.txt"
    data_collected = np.loadtxt(filename)
    X = data_collected[:, COLUMN]
    return X     
# -----------------------------------------------
def BETA_PDF(MIN_X, MAX_X, a, b):
    import numpy as np
    return MIN_X + (MAX_X - MIN_X) * np.random.beta(a, b)
# -----------------------------------------------
def PLOT3D(X, Y, Z, XLABEL, YLABEL, ZLABEL, TITLE):
    import plotly.graph_objects as go
    # Create a 3D scatter plot
    fig = go.Figure(data=[go.Scatter3d(x=X, y=Y, z=Z, mode='markers', marker=dict(size=5, color=Z))])
    fig.update_layout(scene=dict(xaxis_title=XLABEL, yaxis_title=YLABEL, zaxis_title=ZLABEL), title=TITLE)
    fig.show()
# -----------------------------------------------    
def HISTOGRAM_BOXPLOT_PLOTLY( DATA, XLABEL='X', TITLE='A', COLOR='cyan'):
    # Plotting histogram and boxplot
    import plotly.express as px
    fig = px.histogram(x=DATA, marginal="box", color_discrete_sequence=[COLOR])
    fig.update_layout(title=TITLE, xaxis_title=XLABEL, yaxis_title="Frequency")
    fig.show() 
# -----------------------------------------------     
# Create a scatter plot
def PLOT_SCATTER(X, Y , XLABEL, YLABEL, TITLE, COLOR, LOG):
    import matplotlib.pyplot as plt
    # Calculate linear regression parameters
    import numpy as np
    coefficients = np.polyfit(X, Y, 1)
    a, b = coefficients
    y = [];yy = [];
    for i in range(len(X)):
        y.append(a * X[i] + b)
        yy.append(Y[i] - y[-1])
    y = np.array(y)    
    yy = np.array(yy) 
    # Calculate TSS
    Y_mean = np.mean(Y)
    TSS = np.sum((Y - Y_mean) ** 2)
    # Calculate RSS
    RSS = np.sum(yy ** 2)
    # Calculate R-squared
    R_squared = 1 - (RSS / TSS)
    #print(f"R-squared value: {R_squared:.4f}")
    plt.figure(figsize=(10,6))
    plt.scatter(X, Y, color=COLOR, marker='o', label='Data')
    # Add labels and title
    plt.xlabel(XLABEL)
    plt.ylabel(YLABEL)
    # Add the linear regression line
    plt.plot(X, y, color='black', label=f'y = {a:.2f}x + {b:.2f} - R^2 = {R_squared:.3f}')
    plt.title(TITLE)
    plt.grid(True)
    plt.legend()
    if LOG == 1:
        plt.semilogx();plt.semilogy();
    plt.show()

def plot_scatter_plotly(X, Y, XLABEL, YLABEL, TITLE, COLOR):
    import plotly.express as px
    fig = px.scatter(x=X, y=Y, color_discrete_sequence=[COLOR], labels={XLABEL: XLABEL, YLABEL: YLABEL})
    fig.update_layout(title=TITLE, xaxis_type='log', yaxis_type='log')
    fig.show() 
# ----------------------------------------------- 
def PLOT_HEATMAP(df):
    import plotly.figure_factory as ff
    # Calculate the correlation matrix
    corr_matrix = df.corr()

    # Create a correlation heatmap
    fig = ff.create_annotated_heatmap(
        z=corr_matrix.values,
        x=list(corr_matrix.columns),
        y=list(corr_matrix.index),
        annotation_text=corr_matrix.round(5).values,
        showscale=True,
        colorscale='Viridis'
    )

    # Update layout
    fig.update_layout(
        title='Correlation Heatmap',
        xaxis=dict(title='Variable'),
        yaxis=dict(title='Variable'),
        width=800, height=700
    )

    fig.show()
# -----------------------------------------------
def BILNEAR_CURVE(Cur, Mom):
    import numpy as np
    # bilinear fitting
    SIZE = len(Mom)
    hh = np.zeros(SIZE-1)
    Aa = np.zeros(SIZE-1)
    for i in range(SIZE-1):
        hh[i] = Cur[i+1] - Cur[i]
        Aa[i] = (Mom[i] + Mom[i+1]) * 0.5 * hh[i]

    Area = sum(Aa)
    k0 = Mom[2] / Cur[2]
    fiy = (Mom[i+1] * max(Cur) * 0.5 - Area) / (Mom[i+1] * 0.5 - k0 * max(Cur) * 0.5)
    My = k0 * fiy
    X = np.array([0, fiy, max(Cur)])
    Y = np.array([0, My, Mom[i+1]])
    """
    print('+==========================+')
    print('=   Analysis curve fitted =')
    print('     Disp    BaserShear')
    print('----------------------------')
    print(np.column_stack((X.T, Y.T)))
    print('+==========================+')
    """
    # EI and Ductility_Rito of Unconfined Section
    Elastic_ST = Y[1] / X[1]
    Plastic_ST = Y[2] / X[2]
    Tangent_ST = (Y[2] - Y[1]) / (X[2] - X[1])
    Ductility_Rito = X[2] / X[1]
    Over_Strength_Factor = Y[2] / Y[1]
    """
    print('+--------------------------------------------------------------+')
    print(f' Elastic Stiffness :             {Elastic_ST:.2f}')
    print(f' Plastic Stiffness :             {Plastic_ST:.2f}')
    print(f' Tangent Stiffness :             {Tangent_ST:.2f}')
    print(f' Section Ductility Ratio :       {Ductility_Rito:.2f}')
    print(f' Section Over Strength Factor:   {Over_Strength_Factor:.2f}')
    print('+--------------------------------------------------------------+')
    """
    return X, Y, Elastic_ST, Plastic_ST, Tangent_ST, Ductility_Rito, Over_Strength_Factor
# -----------------------------------------------
def OUTLIER_DATA(NameFiles, I):
    # Remove The Outlier Data
    import numpy as np
    filename = f"C:\OPENSEESPY_SALAR\OPENSEESPY_DATA\\{NameFiles}_{I}.txt"
    D = len(np.loadtxt(filename))
    if D >= 4:
        #print(f"Number of rows: {D}")
        return 1

In [ ]:
# pip install openseespy

In [ ]:
# OUTPUT DATA ADDRESS:
SALAR_DIR = 'C:/OPENSEESPY_SALAR/OPENSEESPY_DATA/';

In [ ]:
#
#    ^Y
#    |
#    2       __ 
#    |          | 
#    |          |
#    |          |
#  (1)       LCol
#    |          |
#    |          |
#    |          |
#  =1=      _|_  -------->X
#

# SET UP ----------------------------------------------------------------------------


def PUSHOVER_ANALYSIS(hw_COL, tw_COL,tf_COL, bf_COL, hw_BEAM, tw_BEAM, tf_BEAM, bf_BEAM, H1, H2, L1, N, ND, Weight, UL, DMAX, I):
    NN = int(N / 4) # 4 ELEMENTS
    import openseespy.opensees as op
    op.wipe()
    op.model('basic', '-ndm', 2, '-ndf', 3) 
    PCol =Weight  # nodal dead-load weight per column
    g =  9810 # mm/s^2
    Mass =  PCol/g
    h1 = H1 / (NN - 1)
    h2 = H2 / (NN - 1)
    bl = (L1 **2 + H2 **2)**0.5  # Beam Length
    l1 = L1 / (NN) 
          
    IDctrlNode = ND ## INCREMENTAL DISPLACEMENT NODE
    IDctrlDOF = 1
    
    # MATERIAL parameters -------------------------------------------------------------------
    IDreinf = 1; 				# material ID tag -- steel 
    Fy = 240			# STEEL yield stress
    Cy = 0.0012			# STEEL yield stress
    Es = Fy/Cy				# modulus of steel
    Bs = 0.01				# strain-hardening ratio 
    R0 = 18.0				# control the transition from elastic to plastic branches
    cR1 = 0.925				# control the transition from elastic to plastic branches
    cR2 = 0.15				# control the transition from elastic to plastic branches
    op.uniaxialMaterial('Steel02', IDreinf, Fy, Es, Bs, R0,cR1,cR2) # build reinforcement material
    # ---------------------------------------------------------------------------------------
    hwbot_COL = 0.5 * hw_COL # Bottom column web height
    # nodal coordinates:
    op.node(1, 0.0, 0.0) # node#, X, Y
    op.node(N, L1 * 2, 0.0) # node#, X, Y
    # Single point constraints -- Boundary Conditions
    op.fix(1, 1, 1, 1) # node DX DY RZ
    op.fix(N, 1, 1, 1) # node DX DY RZ
    
    # ----------------------------------------------------------
    # COLUMN 01:
    hwbot_COL = 0.5 * hw_COL # Bottom column web height - SECTION 01
    for i in range(2, NN + 1, 1):
        pp = i-1
        x = 0; y = h1 * pp;
        op.node(i, x, y)
        ColSecTag = pp			# assign a tag number to the column section
        ColTransfTag = pp
        eleTag = pp

        # FIBER SECTION properties -------------------------------------------------------------
        # symmetric section
        #                        y
        #                        ^
        #                        |     
        #              _____________________    --   --
        #             |_________   ________|    |    -- tf
        #                      |  |             |
        #                      |  |             |
        #    z <---       hw   |tw|             H
        #                      |  |             |
        #              ________|  |_________    |
        #             |____________________|    |    -- tf
        #                                      --    --
        #             |-------- bf --------|
        #
        # STEEL I SECTION: 
        HW_COL =  hwbot_COL + ((hw_COL - hwbot_COL) / H1) * h1 * (i-1) # Varying web height of nonprismatic section - SECTION 02
        #print(i, x, y, i-1, i, HW_COL)
        coverY = (HW_COL + tf_COL) / 2.0
        coverZ = tw_COL / 2.0
        coreY = coverY - tf_COL
        coreZ02 = bf_COL / 2.0

        nfCoreY = 15;			# number of fibers for steel in y-direction
        nfCoreZ = 5;			# number of fibers for steel in z-direction


        op.section('Fiber', ColSecTag)
        # Define the core patch
        op.patch('quad', IDreinf, nfCoreZ, nfCoreY, -coverY, coreZ02, -coverY, -coreZ02, -coreY,-coreZ02, coreY, coreZ02) # BOTTOM FLANGE
        op.patch('quad', IDreinf, nfCoreZ, nfCoreY, -coreY,coverZ, -coreY,-coverZ, coreY,-coverZ, coreY, coverZ) # MIDDLE WEB
        op.patch('quad', IDreinf, nfCoreZ, nfCoreY, coreY, coreZ02, -coreY, coreZ02, coverY,-coreZ02, coreY, coreZ02) # TOP FLANGE

        op.geomTransf('Linear', ColTransfTag)
        numIntgrPts = 5
        op.element('nonlinearBeamColumn', eleTag, i-1, i, numIntgrPts, ColSecTag, 1)
    # ----------------------------------------------------------
    # BEAM 01:
    hwbot_BEAM = 0.5 * hw_BEAM # j section beam web height  - SECTION 04
    for i in range(NN + 1, 2 * NN + 1, 1):
        zz = i - NN
        x = l1 * zz; y = H1 + h2 * zz;
        op.node(i, x, y)
        ColSecTag = i-1			# assign a tag number to the column section
        ColTransfTag = i-1
        eleTag = i-1

        # FIBER SECTION properties -------------------------------------------------------------
        # symmetric section
        #                        y
        #                        ^
        #                        |     
        #              _____________________    --   --
        #             |_________   ________|    |    -- tf
        #                      |  |             |
        #                      |  |             |
        #    z <---       hw   |tw|             H
        #                      |  |             |
        #              ________|  |_________    |
        #             |____________________|    |    -- tf
        #                                      --    --
        #             |-------- bf --------|
        #
        # STEEL I SECTION: 
        HW_BEAM =  hw_BEAM + ((hwbot_BEAM - hw_BEAM) / L1) * l1 * zz # Varying web height of nonprismatic section - SECTION 03
        #print(i, x, y, i-1, i, HW_BEAM)
        coverY = (HW_BEAM + tf_BEAM) / 2.0
        coverZ = tw_BEAM / 2.0
        coreY = coverY - tf_BEAM
        coreZ02 = bf_BEAM / 2.0

        nfCoreY = 15;			# number of fibers for steel in y-direction
        nfCoreZ = 5;			# number of fibers for steel in z-direction


        op.section('Fiber', ColSecTag)
        # Define the core patch
        op.patch('quad', IDreinf, nfCoreZ, nfCoreY, -coverY, coreZ02, -coverY, -coreZ02, -coreY,-coreZ02, coreY, coreZ02) # BOTTOM FLANGE
        op.patch('quad', IDreinf, nfCoreZ, nfCoreY, -coreY,coverZ, -coreY,-coverZ, coreY,-coverZ, coreY, coverZ) # MIDDLE WEB
        op.patch('quad', IDreinf, nfCoreZ, nfCoreY, coreY, coreZ02, -coreY, coreZ02, coverY,-coreZ02, coreY, coreZ02) # TOP FLANGE

        op.geomTransf('Linear', ColTransfTag)
        numIntgrPts = 5
        op.element('nonlinearBeamColumn', eleTag, i-1, i, numIntgrPts, ColSecTag, 1) 
    # ----------------------------------------------------------
    # COLUMN 02:
    hwbot_COL = 0.5 * hw_COL # Bottom column web height
    for i in range(N-1, 3 * NN, -1):
        pp = i - 1
        zz = N - i
        #print(pp)
        x = L1 * 2; y = h1 * zz;
        #print(x, y)
        op.node(i, x, y)
        ColSecTag = i			# assign a tag number to the column section
        ColTransfTag = i
        eleTag = i

        # FIBER SECTION properties -------------------------------------------------------------
        # symmetric section
        #                        y
        #                        ^
        #                        |     
        #              _____________________    --   --
        #             |_________   ________|    |    -- tf
        #                      |  |             |
        #                      |  |             |
        #    z <---       hw   |tw|             H
        #                      |  |             |
        #              ________|  |_________    |
        #             |____________________|    |    -- tf
        #                                      --    --
        #             |-------- bf --------|
        #
        # STEEL I SECTION: 
        HW_COL =  hwbot_COL + ((hw_COL - hwbot_COL) / H1) * h1 * (N - i) # Varying web height of nonprismatic section
        #print(i, x, y, i+1, i, HW_COL)
        coverY = (HW_COL + tf_COL) / 2.0
        coverZ = tw_COL / 2.0
        coreY = coverY - tf_COL
        coreZ02 = bf_COL / 2.0

        nfCoreY = 15;			# number of fibers for steel in y-direction
        nfCoreZ = 5;			# number of fibers for steel in z-direction


        op.section('Fiber', ColSecTag)
        # Define the core patch
        op.patch('quad', IDreinf, nfCoreZ, nfCoreY, -coverY, coreZ02, -coverY, -coreZ02, -coreY,-coreZ02, coreY, coreZ02) # BOTTOM FLANGE
        op.patch('quad', IDreinf, nfCoreZ, nfCoreY, -coreY,coverZ, -coreY,-coverZ, coreY,-coverZ, coreY, coverZ) # MIDDLE WEB
        op.patch('quad', IDreinf, nfCoreZ, nfCoreY, coreY, coreZ02, -coreY, coreZ02, coverY,-coreZ02, coreY, coreZ02) # TOP FLANGE

        op.geomTransf('Linear', ColTransfTag)
        numIntgrPts = 5
        op.element('nonlinearBeamColumn', eleTag, i+1, i, numIntgrPts, ColSecTag, 1)  
    # ----------------------------------------------------------
    # BEAM 02:
    hwbot_BEAM = 0.5 * hw_BEAM # j section beam web height
    for i in range(3 * NN, 2 * NN, -1):
        zz = 3 * NN + 1 - i
        x = 2 * L1 - l1 * zz; y = H1 + h2 * zz;
        op.node(i, x, y) 
        ColSecTag = i			# assign a tag number to the column section
        ColTransfTag = i
        eleTag = i


        # FIBER SECTION properties -------------------------------------------------------------
        # symmetric section
        #                        y
        #                        ^
        #                        |     
        #              _____________________    --   --
        #             |_________   ________|    |    -- tf
        #                      |  |             |
        #                      |  |             |
        #    z <---       hw   |tw|             H
        #                      |  |             |
        #              ________|  |_________    |
        #             |____________________|    |    -- tf
        #                                      --    --
        #             |-------- bf --------|
        #
        # STEEL I SECTION: 
        HW_BEAM =  hw_BEAM + ((hwbot_BEAM - hw_BEAM) / L1) * l1 * zz # Varying web height of nonprismatic section
        #print(i, x, y, i+1, i, HW_BEAM)
        coverY = (HW_BEAM + tf_BEAM) / 2.0
        coverZ = tw_BEAM / 2.0
        coreY = coverY - tf_BEAM
        coreZ02 = bf_BEAM / 2.0

        nfCoreY = 15;			# number of fibers for steel in y-direction
        nfCoreZ = 5;			# number of fibers for steel in z-direction

        op.section('Fiber', ColSecTag)
        # Define the core patch
        op.patch('quad', IDreinf, nfCoreZ, nfCoreY, -coverY, coreZ02, -coverY, -coreZ02, -coreY,-coreZ02, coreY, coreZ02) # BOTTOM FLANGE
        op.patch('quad', IDreinf, nfCoreZ, nfCoreY, -coreY,coverZ, -coreY,-coverZ, coreY,-coverZ, coreY, coverZ) # MIDDLE WEB
        op.patch('quad', IDreinf, nfCoreZ, nfCoreY, coreY, coreZ02, -coreY, coreZ02, coverY,-coreZ02, coreY, coreZ02) # TOP FLANGE

        op.geomTransf('Linear', ColTransfTag)
        numIntgrPts = 5
        op.element('nonlinearBeamColumn', eleTag, i+1, i, numIntgrPts, ColSecTag, 1)  
        
        
    op.element('nonlinearBeamColumn', 200, 102, 100, numIntgrPts, ColSecTag, 1) ## THIS ELEMENT IS FOR CONNECTION 2 SEPARATE ELEMENTS TO EACH OTHER  
    #import InelasticFiberSection
    op.recorder('Node', '-file', f"{SALAR_DIR}DTH_{I}.txt",'-time', '-node', ND, '-dof', 1,2,3, 'disp')# Displacement Time History
    op.recorder('Node', '-file', f"{SALAR_DIR}BTH_{I}.txt",'-time', '-node', 1, '-dof', 1,2,3, 'reaction')# Base Shear Time History

    # node#, Mx My Mz, Mass=Weight/g, neglect rotational inertia at nodes
    #op.mass(2*NN, Mass, 1e-9, 0.0)
    #defining gravity loads
    op.timeSeries('Linear', 1)
    op.pattern('Plain', 1, 1)
    for i in range(NN + 1, 2 * NN + 1, 1): # CREATE UNIFORM LOADS FOR BEAM 01
        op.eleLoad('-ele', i-1,'-type', '-beamUniform', UL, 0.0) # uniformly-distributed load
    for i in range(3 * NN, 2 * NN, -1):# CREATE UNIFORM LOADS FOR BEAM 02
        op.eleLoad('-ele', i,'-type', '-beamUniform', UL, 0.0) # uniformly-distributed load
    
    Tol = 1e-8 # convergence tolerance for test
    Iter = 1000# convergence iteration for test
    NstepGravity = 10
    DGravity = 1 / NstepGravity
    op.integrator('LoadControl', DGravity) # determine the next time step for an analysis
    op.numberer('Plain') # renumber dof's to minimize band-width (optimization), if you want to
    op.system('BandGeneral') # how to store and solve the system of equations in the analysis
    op.constraints('Plain') # how it handles boundary conditions
    op.test('NormDispIncr', Tol, Iter) # determine if convergence has been achieved at the end of an iteration step
    op.algorithm('Newton') # use Newton's solution algorithm: updates tangent stiffness at every iteration
    op.analysis('Static') # define type of analysis static or transient
    op.analyze(NstepGravity) # apply gravity

    op.loadConst('-time', 0.0) #maintain constant gravity loads and reset time to zero
    print('Model Built')
    
    Dincr = 0.001 * DMAX
    Hload = 1#Weight
    maxNumIter = 1000
    tol = 1e-8

    op.timeSeries('Linear', 2)
    op.pattern('Plain', 200, 2)
    op.load(ND, Hload, 0.0, 0.0)

    op.wipeAnalysis()
    op.constraints('Plain')
    op.numberer('Plain')
    op.system('BandGeneral')
    op.test('EnergyIncr', Tol, maxNumIter)
    op.algorithm('Newton')

    op.integrator('DisplacementControl', IDctrlNode, IDctrlDOF, Dincr)
    op.analysis('Static')


    Nsteps =  int(DMAX/ Dincr)

    ok = op.analyze(Nsteps)
    #print(ok)

    # for gravity analysis, load control is fine, 0.1 is the load factor increment (http://opensees.berkeley.edu/wiki/index.php/Load_Control)

    test = {1:'NormDispIncr', 2: 'RelativeEnergyIncr', 4: 'RelativeNormUnbalance',5: 'RelativeNormDispIncr', 6: 'NormUnbalance'}
    algorithm = {1:'KrylovNewton', 2: 'SecantNewton' , 4: 'RaphsonNewton',5: 'PeriodicNewton', 6: 'BFGS', 7: 'Broyden', 8: 'NewtonLineSearch'}

    for i in test:
        for j in algorithm:

            if ok != 0:
                if j < 4:
                    op.algorithm(algorithm[j], '-initial')

                else:
                    op.algorithm(algorithm[j])

                op.test(test[i], Tol, 1000)
                ok = op.analyze(Nsteps)                            
                #print(test[i], algorithm[j], ok)             
                if ok == 0:
                    break
            else:
                continue

    #u2 = op.nodeDisp(2, 1)
    #print("u2 = ", u2)
    print('Pushover Done')
    op.wipe()
    


In [ ]:
### ---------------------------------------------------------------------
###    NONPRISMATIC I SECTION WEB HEIGHT SENSITIVITY ANALYSIS
### ---------------------------------------------------------------------

# define section geometry
hw_COL_MIN = 280 # [mm] Minimum Column Section Web Height 
hw_COL_MAX = 350 # [mm] Maximum Column Section Web Height 
hw_BEAM_MIN = 80 # [mm] Minimum Beam Section Web Height 
hw_BEAM_MAX = 150 # [mm] Maximum Beam Section Web Height 


# define section geometry
H1 = 3000.0 # [mm] Column length
H2 = 1000.0 # [mm] Beam length
L1 = 5000.0 # [mm] Beam length

tw_COL = 10 # [mm] Section Web Thickness
tf_COL = 10 # [mm] Section Flange Thickness
bf_COL = 110 # [mm] Section Flange Width

tw_BEAM = 10 # [mm] Section Web Thickness
tf_BEAM = 10 # [mm] Section Flange Thickness
bf_BEAM = 110 # [mm] Section Flange Width

N = 200 # STRUCTURES NODES COUNT
ND = 150 # NODE NUMBER APPLIED INCREMENTAL DISPLACEMENT

Weight = 100000.0 # [N] superstructure weight
ul = -10 #[N/mm] Uniform Distributed Loads
DMAX = 300 # [mm] Max. Pushover Incremental Displacement

HWC = [] # Retrofit Column length Array
HWB = [] # Retrofit Column Section Pipe Thickness Array

# monitor cpu time
import time
t = time.localtime()
current_time = time.strftime("%H:%M:%S", t)
print(f"Current time (HH:MM:SS): {current_time}\n\n")

NUM_ITERATION = 150
for i in range(NUM_ITERATION):
    hwc = BETA_PDF(hw_COL_MIN, hw_COL_MAX, 1, 1)# Random Column Section Web Height 
    hwb = BETA_PDF(hw_BEAM_MIN, hw_BEAM_MAX, 1, 1)# Random Beam Section Web Height
    HWC.append(hwc); HWB.append(hwb);
    PUSHOVER_ANALYSIS(hwc, tw_COL,tf_COL, bf_COL, hwb, tw_BEAM, tf_BEAM, bf_BEAM, H1, H2, L1, N, ND, Weight, ul, DMAX, i)
    print(f' {i+1} hwc {hwc:.4f} hwb {hwb:.4f}')
    
    
    
t = time.localtime()
current_time = time.strftime("%H:%M:%S", t)
print(f"Current time (HH:MM:SS): {current_time}\n\n")

In [ ]:
# LOADING DATAS
DATA_FILE01 ='C:\OPENSEESPY_SALAR\OPENSEESPY_DATA\DTH'  # DEFORMATION
DATA_FILE02 ='C:\OPENSEESPY_SALAR\OPENSEESPY_DATA\DTH'  # MAX BASE SHEAR

import time
# monitor cpu time
import time
starttime = time.process_time()

dd = [] # Displacement array 1 
bs = [] # Base shear array 1
osf = [] # Over Stregth Factor
dr = [] # Ductility Rito
pei = [] # Plastic Stiffness
eei = [] # Elastic Stiffness

NUM_ITERATION = 150
for i in range(NUM_ITERATION):
    dd.append(MAXABS_FUN(DATA_FILE01, 1, i))# DIPLACEMENT
    bs.append(MAXABS_FUN(DATA_FILE02, 1, i))# BASE SHEAR
    disp = OUTPUT_SECOND_COLUMN('DTH', 1, i) # Reading Disp from Text file
    base = abs(OUTPUT_SECOND_COLUMN('BTH', 1, i)) # Reading base shear from Text file
    _, _, Elastic_ST, Plastic_ST, _, Ductility_Rito, Over_Strength_Factor = BILNEAR_CURVE(disp, base)
    osf.append(Over_Strength_Factor)
    dr.append(Ductility_Rito)
    pei.append(Plastic_ST)
    eei.append(Elastic_ST)
    print('iter ',i+1)
    
print('ALL DATAS READED') 


totaltime = time.process_time() - starttime
print(f'\nTotal time (s): {totaltime:.4f} \n\n')

In [ ]:
HISROGRAM_BOXPLOT(HWC, HISTO_COLOR='yellow', LABEL='Column Section Web Height')

In [ ]:
HISROGRAM_BOXPLOT(HWB, HISTO_COLOR='lightblue', LABEL='Beam Section Web Height')

In [ ]:
HISROGRAM_BOXPLOT(dd, HISTO_COLOR='pink', LABEL='MAx. Abs. Displacement DOF [448]')

In [ ]:
HISROGRAM_BOXPLOT(osf, HISTO_COLOR='lime', LABEL='Over Stregth Factor')

In [ ]:
HISROGRAM_BOXPLOT(dr, HISTO_COLOR='lightgray', LABEL='Ductility Rito')

In [ ]:
HISROGRAM_BOXPLOT(pei, HISTO_COLOR='purple', LABEL='Plastic Structural Stiffness')

In [ ]:
HISROGRAM_BOXPLOT(eei, HISTO_COLOR='green', LABEL='Elastic Structural Stiffness')

In [ ]:
PLOT_SCATTER(HWC, dd, XLABEL= 'Column Section Web Height', YLABEL='MAx. Abs. Displacement DOF [448]', TITLE='Column Section Web Height & MAx. Abs. Displacement', COLOR = 'olive', LOG = 0) 
#plot_scatter_plotly(HWC, dd, XLABEL= 'Column Section Web Height', YLABEL='MAx. Abs. Displacement', TITLE='Column Section Web Height & MAx. Abs. Displacement', COLOR = 'olive')

In [ ]:
PLOT_SCATTER(HWB, dd, XLABEL= 'Beam Section Web Height', YLABEL='MAx. Abs. Displacement DOF [448]', TITLE='Beam Section Web Height & MAx. Abs. Displacement', COLOR = 'silver', LOG = 0) 
#plot_scatter_plotly(HWB, dd, XLABEL= 'Beam Section Web Height', YLABEL='MAx. Abs. Displacement', TITLE='Beam Section Web Height & MAx. Abs. Displacement', COLOR = 'silver')

In [ ]:
PLOT_SCATTER(HWC, osf, XLABEL= 'Column Section Web Height', YLABEL='Over Strength Factor', TITLE='Column Section Web Height & Over Strength Factor', COLOR = 'orange', LOG = 0) 
#plot_scatter_plotly(HWC, osf, XLABEL= 'Beam Section Web Height', YLABEL='Over Strength Factor', TITLE='Column Section Web Height & Over Strength Factor', COLOR = 'orange')

In [ ]:
PLOT_SCATTER(HWC, dr, XLABEL= 'Column Section Web Height', YLABEL='Ductility Rito', TITLE='Column Section Web Height & Ductility Rito', COLOR = 'purple', LOG = 0) 
#plot_scatter_plotly(HWC, dr, XLABEL= 'Beam Section Web Height', YLABEL='Ductility Rito', TITLE='Column Section Web Height & Ductility Rito', COLOR = 'purple')

In [ ]:
PLOT_SCATTER(HWC, pei, XLABEL= 'Column Section Web Height', YLABEL='Plastic Structural Stiffness', TITLE='Column Section Web Height & Plastic Structural Stiffness', COLOR = 'yellow', LOG = 0) 
#plot_scatter_plotly(HWC, pei, XLABEL= 'Beam Section Web Height', YLABEL='Plastic Structural Stiffness', TITLE='Column Section Web Height & Plastic Structural Stiffness', COLOR = 'yellow')

In [ ]:
PLOT_SCATTER(HWC, eei, XLABEL= 'Column Section Web Height', YLABEL='Elastic Structural Stiffness', TITLE='Column Section Web Height & Elastic Structural Stiffness', COLOR = 'blue', LOG = 0) 
#plot_scatter_plotly(HWC, eei, XLABEL= 'Beam Section Web Height', YLABEL='Elastic Structural Stiffness', TITLE='Column Section Web Height & Elastic Structural Stiffness', COLOR = 'blue')

In [ ]:
PLOT3D(HWC, HWB, dr, XLABEL='Column Section Web Height', YLABEL='Beam Section Web Height', ZLABEL='Ductility Rito', TITLE='3D CHART')

In [ ]:
PLOT3D(HWC, HWB, osf, XLABEL='Column Section Web Height', YLABEL='Beam Section Web Height', ZLABEL='Over Strength Factor', TITLE='3D CHART')

In [ ]:
PLOT3D(HWC, HWB, pei, XLABEL='Column Section Web Height', YLABEL='Beam Section Web Height', ZLABEL='Plastic Structural Stiffness', TITLE='3D CHART')

In [ ]:
PLOT3D(HWC, HWB, eei, XLABEL='Column Section Web Height', YLABEL='Beam Section Web Height', ZLABEL='Elastic Structural Stiffness', TITLE='3D CHART')

In [ ]:
import pandas as pd
# Create a DataFrame
df = pd.DataFrame({'Column Section Web Height': HWC, 'Beam Section Web Height': HWB,'MAx. Abs. Displacement DOF [448]': dd, 'MAx. Abs. Base Shear DOF [1]': bs,'Over Strength Factor':osf,'Ductility Rito':dr,'Plastic Structural Stiffness':pei,'Elastic Structural Stiffness':eei})
print(df)
# PLOT HEATMAP FOR CORRELATION 
PLOT_HEATMAP(df) 

In [ ]:
### Multiple Regression Model

import statsmodels.api as sm
# Add a constant term for the intercept
X = sm.add_constant(df[['Column Section Web Height', 'Beam Section Web Height', 'Over Strength Factor', 'Ductility Rito', 'Plastic Structural Stiffness', 'Elastic Structural Stiffness']])

# Fit the multiple regression model
model = sm.OLS(df['MAx. Abs. Displacement DOF [448]'], X).fit()

# Print the summary
print(model.summary())
